<a href="https://colab.research.google.com/github/viniciusrpb/cic0269_natural_language_processing/blob/main/statistical_tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testes Estatísticos para Aprendizado de Máquina

Com base no artigo [*Statistical Comparisons of Classifiers over Multiple Data Sets*](https://www.jmlr.org/papers/volume7/demsar06a/demsar06a.pdf), de Janez Demsar, 2006.

### Teste de Hipótese pelo Teste de Wilcoxon

Comparar o desempenho de dois classificadores em um mesmo dataset utilizando o *Wilcoxon Signed-Rank Test*.



In [1]:
from scipy.stats import wilcoxon, friedmanchisquare, rankdata
from sklearn.datasets import load_iris
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score

Seja o conjunto de dados Iris. Vamos pegar os dados:

In [2]:
iris = load_iris()
X = iris.data
y = iris.target

Pré-processamento

In [3]:
scaler = StandardScaler()

### Procedimento Geral para Testes de Hipóteses

1.   A partir do contexto do problema, identifique o parâmetro de interesse;
2.   Estabeleça a hipótese nula $H_0$;
3.   Especifique uma hipótese alternativa $H_1$;
4.   Escolha um nível de significância $\alpha$;
5.   Determine uma estatística apropriada de teste;
6.   Estabeleça a região de rejeição para a estatística;
7.   Calcule quaisquer grandezas amostrais necessárias,substitua-as na equação para a estatística de teste e calcule aquele valor;
8.   Decida se $H_0$ deve ou não ser rejeitada.

## Teste de Wilcoxon

Primeiramente, precisamos formular uma hipótese estatística, que é uma afirmação sobre os parâmetros de uma ou mais populações. Em aprendizado de máquina, o que seriam essas populações?

Sabemos que a ideia é comparar os classificadores Naive Bayes e SVM e verificar se seus desempenhos (acertos na classificação) permitem identificar desempenhos similares ou não entre eles. Nesse caso, podemos estabelecer como hipótese nula $H_0$ que seus desempenhos (medidos pela F1-Score) são equivalentes.

Vamos repetir uma amostragem aleatória 20 vezes (pode ser um K-Fold Cross Validation também):


In [ ]:
f1_scores = []

for i in range(0,21):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.34, stratify=y, random_state=i)

    # SVM 
    svm  = SVC(gamma='auto')

    scaler.fit(X_train)

    X_train_std = scaler.transform(X_train)

    svm.fit(X_train_std, y_train)

    X_test_std = scaler.transform(X_test)

    y_svm_pred = svm.predict(X_test_std)

    # Naive Bayes
    gnb = GaussianNB()
    
    gnb.fit(X_train_std, y_train)

    y_gnb_pred = gnb.predict(X_test_std)

    print(y_test)

    print(y_svm_pred)

    f1_svm = f1_score(y_test, y_svm_pred, average='macro')

    f1_gnb = f1_score(y_test, y_gnb_pred, average='macro')

    f1_scores.append([f1_svm,f1_gnb])

In [17]:
f1_scores = np.array(f1_scores)

Verificando as F1-Scores para cada amostragem aleatória:

In [18]:
df = pd.DataFrame(f1_scores, columns = ['SVM', 'Gaussian Naive-Bayes'])
df

,SVM,Gaussian Naive-Bayes
0,0.960784,0.960784
1,0.961874,0.980890
2,1.000000,1.000000
3,0.923656,0.961874
4,0.923747,0.961874
5,0.923747,0.942857
6,0.923246,0.980952
7,0.941126,0.960784
8,0.884416,0.881944
9,0.961874,0.942857


In [19]:
wilcoxon(f1_scores[:,0], f1_scores[:,1], zero_method='zsplit')

/usr/local/lib/python3.9/dist-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


WilcoxonResult(statistic=71.0, pvalue=0.12032389796169081)

O valor $\rho$ de um teste de hipótese é o menor nível de significância que conduz à rejeição da hipótese nula $H_0$, com os dados fornecidos.


O teste de Wilcoxon retornou um $\rho$-valor próximo a $0.12$. Se o nível de significância for igual a $0.05$, podemos concluir que os desempenhos do SVM e o NB são equivalentes.

## Teste de Nemenyi